In [9]:
import pathlib
import pandas as pd
import shutil
import logging

from idrstream.DP_idr import DeepProfilerRun

In [10]:
data_to_process = pd.read_csv("example_files/data_to_process.tsv", sep="\t", index_col=0)
# remove A1 wells because of irregular illumination
data_to_process = data_to_process[data_to_process["Well"] != "A1"]
data_to_process = data_to_process.reset_index(drop=True)
data_to_process

,Well,Frames,Well Number,Control Type,Original Gene Target,Plate_Map_Name,Gene_Replicate,Site,DNA
0,A4,50,4,positive control,ENSG00000149503,LT0001_02_4,1,1,LT0001_02/LT0001_02_4_50.tif
1,A15,50,15,negative control,negative control,LT0001_02_15,1,1,LT0001_02/LT0001_02_15_50.tif
2,B2,50,26,negative control,negative control,LT0001_02_26,1,1,LT0001_02/LT0001_02_26_50.tif
3,C1,50,49,positive control,ENSG00000149503,LT0001_02_49,1,1,LT0001_02/LT0001_02_49_50.tif
4,C4,50,52,positive control,ENSG00000149503,LT0001_02_52,1,1,LT0001_02/LT0001_02_52_50.tif
...,...,...,...,...,...,...,...,...,...
6738,O13,50,349,positive control,KIF11,LT0603_06_349,1,1,LT0603_06/LT0603_06_349_50.tif
6739,O16,50,352,negative control,negative control,LT0603_06_352,1,1,LT0603_06/LT0603_06_352_50.tif
6740,P3,50,363,negative control,negative control,LT0603_06_363,1,1,LT0603_06/LT0603_06_363_50.tif
6741,P21,50,381,positive control,COPB,LT0603_06_381,1,1,LT0603_06/LT0603_06_381_50.tif


In [11]:
idr_id = "idr0013"
tmp_dir = pathlib.Path("tmp/")
final_data_dir = pathlib.Path("mitocheck_control_features/")
try:
    shutil.rmtree(tmp_dir)
    # shutil.rmtree(final_data_dir)
    pass
except:
    print("No files to remove")

stream = DeepProfilerRun(idr_id, tmp_dir, final_data_dir, log='dp_idrstream.log')

In [12]:
aspera_path = pathlib.Path("/home/jenna/.aspera/ascli/sdk/ascp")
aspera_key_path = pathlib.Path("example_files/asperaweb_id_dsa.openssh")
screens_path = pathlib.Path("example_files/idr0013-screenA-plates.tsv")
save_dir = pathlib.Path("data/")

stream.init_downloader(aspera_path, aspera_key_path, screens_path)

In [13]:
#currently causing major issues... again
fiji_path = pathlib.Path("/home/jenna/Desktop/test/Fiji.app")
stream.init_preprocessor(fiji_path)

In [14]:
nuclei_model_specs = {
            "model_type": "cyto",
            "channels": [0, 0],
            "diameter": 0,
            "flow_threshold": 0.8,
            "cellprob_threshold": 0,
            "remove_edge_masks": True,
        }
stream.init_segmentor(nuclei_model_specs)

>>> GPU activated? 1


In [15]:
config_path = pathlib.Path("example_files/DP_files/mitocheck_profiling_config.json")
checkpoint_path = pathlib.Path("example_files/DP_files/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment.h5")
stream.copy_DP_files(config_path, checkpoint_path)

In [18]:
stream.run_dp_stream(data_to_process, batch_size=3, start_batch=0, batch_nums=[0])